In [1]:
from tqdm import tqdm, trange
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
import numpy as np
import warnings
import matplotlib.pyplot as plt


warnings.filterwarnings('ignore')
rescale_size = 244
batch_size = 64

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cpu')

In [3]:
model = torch.load("model.pth", weights_only=False, map_location=device)
model.to(device)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
model.fc

Linear(in_features=512, out_features=2, bias=True)

In [5]:
transform = transforms.Compose([
    transforms.Resize((int(rescale_size * 1.25), int(rescale_size * 1.25))),
    transforms.RandomCrop(rescale_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
import os
from PIL import Image
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.files = sorted([f for f in os.listdir(root) if f.endswith(".png")])
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        path = os.path.join(self.root, self.files[idx])
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, self.files[idx]  # вернем имя файла

In [7]:
test_dataset = TestDataset("/mnt/data/new_datasets/test_new", transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
import pandas as pd
model.eval()
results = []
with torch.no_grad():
    for inputs, filenames in tqdm(test_loader, desc="Predicting"):
        if torch.cuda.is_available():
            inputs = inputs.cuda()
        outputs = model(inputs)
        probs = nn.functional.softmax(outputs, dim=1)[:, 1]
        
        for fname, prob in zip(filenames, probs.cpu().numpy()):
            results.append((fname, prob))

Predicting: 100%|██████████| 135/135 [09:40<00:00,  4.30s/it]


In [22]:
result = []
for fname, prob in results:
    result.append({"id": fname.replace(".png", ""), "prediction": prob})

In [23]:
df_main = pd.read_csv("/mnt/data/new_datasets/ml_ozon_counterfeit_new_test.csv")
df_my = pd.DataFrame(result)
df_my['prediction'] = round(df_my['prediction']).astype(int)
df_my['id'] = df_main['id']
df_my.to_csv("/mnt/data/tigran/submission.csv", index=False)
print(f"Predictions saved to /mnt/data/tigran")

Predictions saved to /mnt/data/tigran


In [25]:
df_main = pd.read_csv("/mnt/data/datasets/tables/ml_ozon_сounterfeit_test.csv")
df_my = pd.read_csv("predictions.csv")
df_my['id'] = df_main['id']
df_my.to_csv("/mnt/data/tigran/new_prediction.csv", index=False)
print(f"Predictions saved to /mnt/data/tigran")

Predictions saved to /mnt/data/tigran
